# Breaking MCE with side channel information using Algebraic Cryptanalysis

In [98]:
# This puts the matrix code in canonical form
# This means that we flatten the code to a k x nm generator matrix and echelonize this
# Then we transform it back to a matrix code
# Note that this canonical form is unique for a matrix code, independent of what generators are chosen
def canonical_form(C):
    F = C[0].base_ring()
    n, m, k = (C[0].nrows(), C[0].ncols(), len(C))
    G = matrix(F, k, n*m, [c.list() for c in C])
    pivots = G.pivots()
    assert len(pivots) == k

    # We want to keep track of the transformation into canonical form to ease computation of the side-channel info
    T = matrix(F, k, k, [[G[i][j] for j in pivots] for i in range(k)])^(-1)
    G = T * G

    return [matrix(F, n, m, [G[i][j] for j in range(n*m)]) for i in range(k)], T

In [99]:
# This method applies the transformation (A, B) to the matrix code C and returns the resulting matrix code D
# It is assumed that A, B are square and that all the dimensions check out
# Feel free to use this method in your solution!
def apply_transformation(C, A, B):
    return canonical_form([A * c * B for c in C])

In [100]:
# Generate a random MCE problem with a piece of side channel information
# Doing the construction this way, we are sure that a solution exists
# It might not be unique however!
def generate_MCE_problem_with_hint(F, n, m, k):

    # Generate the secret transformation
    A = random_matrix(F, n, n)
    B = random_matrix(F, m, m)
    while A.is_singular() or B.is_singular(): # Make sure we are working with invertible matrices
        A = random_matrix(F, n, n)
        B = random_matrix(F, m, m)
    
    # Generating C randomly
    C, _ = canonical_form([random_matrix(F, n, m) for _ in range(k)])

    # Apply secret transformation to C to ensure solution
    D, T = apply_transformation(C, A, B)

    # Pick random coordinates for the side-channel information and transform it
    v = random_vector(F, k)
    w = T.transpose()^(-1) * v

    return C, D, v, w

## Your solution

In [101]:
# This method takes two tensors (and some auxiliary information) and returns the transformation that transforms C to D
# Be sure to read instructions.md for some guiding questions
def break_MCE(F, C, D, v, w, n, m, k):
    # Your code here
    return random_matrix(F, n, n), random_matrix(F, m, m)

## Test time

In [102]:
# This method checks whether your solution is correct
# Since the problem might have multiple solutions we just check if it is a valid transformation
# It does not necessarily have to be the same one that was discarded during generation
# In the end, this is what is cryptographically relevant as well!
def check_solution(C, D, A, B):
    D2, _ = apply_transformation(C, A, B)
    return D == D2

In [103]:
# Feel free to pick any parameters you want!
q = 31
n = 5
m = 4
k = 3

# It should not be necessary to edit the calls below
# If you do, make sure you do not accidently use the solution
F = GF(q)
C, D, v, w = generate_MCE_problem_with_hint(F, n, m, k)
A, B = break_MCE(F, C, D, v, w, n, m, k)
check_solution(C, D, A, B)

False